In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import tensorflow as tf

In [2]:
df = pd.read_csv("ferdata.csv")
df['image'][0]

FileNotFoundError: ignored

In [ ]:
X=[]
y=[]

In [ ]:
for index, row in df.iterrows():
  val = row['image'].split(" ")
  try:
    X.append(np.array(val, 'float32'))
    y.append(np.array(row['label']))
  except:
    print(f"error occured at index {index} and row {row}")

In [ ]:
emotion_label_to_text = {0:'Anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
fig = plt.figure(1, (14, 14))

k = 0
for label in sorted(df.label.unique()):
    for j in range(7):
        px = df[df.label==label].image.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

        k += 1
        ax = plt.subplot(7, 7, k)
        ax.imshow(px, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotion_label_to_text[label])
        plt.tight_layout()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)

In [ ]:
print(y_train)

In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 5
width, height = 48,48

In [ ]:
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')
# X_test = np.array(X_test, 'float32')
y_test = np.array(y_test, 'float32')

y_train = np_utils.to_categorical(y_train, num_classes = num_labels)
y_test = np_utils.to_categorical(y_test, num_classes = num_labels)

In [ ]:
y_train.shape

In [ ]:
X_train = X_train/255.0
# X_test = X_test/255.0

X_train = X_train.reshape(X_train.shape[0], 48,48,1)


# X_test = X_test.reshape(X_test.shape[0], 48,48,1)

In [ ]:
#first convolutional layer

model = Sequential()

model.add(Conv2D(128,kernel_size =(3,3), activation='relu',input_shape=(X_train.shape[1:])))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.4))

#second Conv layer


model.add(Conv2D(64 ,kernel_size =(3,3), activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.4))

#third conv layer

model.add(Conv2D(32 ,kernel_size =(3,3), activation='relu'))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

#fully connected neural network
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
print(y_train.shape)

In [ ]:
cnn = model.fit(X_train, y_train, batch_size = batch_size, 
                epochs = epochs,
                verbose =1,
                validation_data = (X_test, y_test),
                shuffle=True,
                callbacks = [callback])